# <center>PROYECTO INDIVIDUAL SINIESTROS VIALES (CABA)<center>

## <center>ETL (Extract, Transform, Load)<center>

### A continuación nos conectaremos con Python, a través de jupyter notebooks para preparar los datos de tal manera que se puedan cargar en un database online: se pasaran los archivos a .csv para imputar facilmente luego en script.sql


### dataset homicidios, se adaptan de tal manera que sea compatible con mysql

In [ ]:
import pandas as pd
import numpy as np

# URLs de los libros de Excel
url_homicidios = "https://cdn.buenosaires.gob.ar/datosabiertos/datasets/transporte-y-obras-publicas/victimas-siniestros-viales/homicidios.xlsx"
url_lesiones = "https://cdn.buenosaires.gob.ar/datosabiertos/datasets/transporte-y-obras-publicas/victimas-siniestros-viales/lesiones.xlsx"

# Leer los libros de Excel y obtener información sobre las hojas
xls_homicidios = pd.ExcelFile(url_homicidios)
xls_lesiones = pd.ExcelFile(url_lesiones)

# Obtener nombres de las hojas y cantidad de hojas
nombres_hojas_homicidios = xls_homicidios.sheet_names
nombres_hojas_lesiones = xls_lesiones.sheet_names
cantidad_hojas_homicidios = len(nombres_hojas_homicidios)
cantidad_hojas_lesiones = len(nombres_hojas_lesiones)
df_homicidios_primera_hoja = pd.read_excel(url_homicidios, sheet_name=nombres_hojas_homicidios[0])
h_victimas = df_homicidios_primera_hoja = pd.read_excel(url_homicidios, sheet_name=nombres_hojas_homicidios[2])
h_hechos = df_homicidios_primera_hoja = pd.read_excel(url_homicidios, sheet_name=nombres_hojas_homicidios[0])

# Lista de columnas en las que se agregarán comillas simples
columnas_con_comillas = ['ID', 'FECHA', 'HORA', 'LUGAR_DEL_HECHO', 'TIPO_DE_CALLE', 
                         'Calle', 'Cruce', 'Dirección Normalizada', 'XY (CABA)', 
                         'PARTICIPANTES', 'VICTIMA', 'ACUSADO']

h_hechos['HH'].replace('SD', 'null', inplace=True)
# Agregar comillas simples a todas las celdas de las columnas especificadas
for columna in columnas_con_comillas:
    h_hechos[columna] = h_hechos[columna].apply(lambda x: f"'{x}'")

# Convertir celdas vacías o que contengan solo espacios en blanco a NaN
h_hechos = h_hechos.replace({'': np.nan, "''": np.nan})
# Reemplazar valores en la columna "pos x" donde aparece "."
h_hechos.loc[h_hechos['Altura'] == '', 'Altura'] = 'null'
# Reemplazar valores NaN en la columna "Altura" por "null"
h_hechos['Altura'].fillna('null', inplace=True)

h_hechos.loc[h_hechos['Altura'] == '.', 'Altura'] = 'null'
h_hechos.loc[h_hechos['pos x'] == '.', 'pos x'] = 'null'
h_hechos.loc[h_hechos['pos y'] == '.', 'pos y'] = 'null'
import pandas as pd

# Crear una lista para almacenar las filas concatenadas
filas_concatenadas = []

# Iterar sobre cada fila del DataFrame
for indice, fila in h_hechos.iterrows():
    # Concatenar los valores de la fila separados por comas
    fila_concatenada = "(" + ", ".join(map(str, fila.values)) + ")"
    # Agregar la fila concatenada a la lista
    filas_concatenadas.append(fila_concatenada)

# Reemplazar la coma final por un punto y coma en la última fila
filas_concatenadas[-1] = filas_concatenadas[-1].replace(",", ";")

# Crear un DataFrame con una sola columna y asignar las filas concatenadas
df_concatenado = pd.DataFrame(filas_concatenadas, columns=['Columna_Unica'])

df_concatenado.to_csv("homicidios_sql.csv", index = False)


### dataset victimas, se adaptan de tal manera que sea compatible con mysql

In [ ]:
# Suponiendo que h_victimas es tu DataFrame
# Reemplazar "SD", "Sd" y "sd" por valores vacíos en todas las columnas del DataFrame
for col in h_victimas.columns:
    h_victimas[col] = h_victimas[col].replace([None], "")
    columnas_con_comillas2 = ['ID_hecho', 'FECHA', 'ROL', 'VICTIMA', 'SEXO','FECHA_FALLECIMIENTO']

# Agregar comillas simples a todas las celdas de las columnas especificadas
for columna in columnas_con_comillas2:
    h_victimas[columna] = h_victimas[columna].apply(lambda x: f"'{x}'")
    h_victimas = h_victimas.drop(columns=["EDAD_GRUPO"])
h_victimas.to_csv("sql_victimas.csv", index= False , encoding = "utf-16")


### dataset poblacion por año, es un excel que tiene la poblacion por año pero en diferentes hojas, se extrajo de la pagina de estadisticas de buenos aires, se extraen todos los df en uno solo, de tal manera de crear una tabla con dato de poblacion por año y se adaptan de tal manera que sea compatible con mysql

In [ ]:
import pandas as pd

# Ruta del archivo Excel
file_path = "https://www.estadisticaciudad.gob.ar/eyc/wp-content/uploads/2021/05/PDE.xlsx"

# Leer el archivo Excel
xls = pd.ExcelFile(file_path)

# Crear un diccionario para almacenar los DataFrames de cada página
dfs = {}

# Iterar sobre cada página del archivo Excel
for sheet_name in xls.sheet_names:
    if sheet_name not in ["Ficha Técnica", "PDE"]:
        # Leer la página del archivo Excel y omitir las primeras 1 filas
        df = pd.read_excel(xls, sheet_name, header=None, skiprows=1)
        # Extraer las filas requeridas (de la 4ta a la 19)
        df = df.iloc[3:17]
        # Agregar una columna con el nombre de la hoja
        df['Hoja'] = sheet_name
        # Almacenar el DataFrame en el diccionario utilizando el nombre de la página como clave
        dfs[sheet_name] = df

# Concatenar todos los DataFrames en uno solo
df_concatenado = pd.concat(dfs.values(), ignore_index=True)

# Renombrar las columnas
df_concatenado.columns = ["comuna", "total", "masculino", "femenino", "superficie_km2", "densidad_poblacion_hab_km2", "anio"]

poblacion = df_concatenado 
poblacion.to_csv("poblacion.csv", index = False)
poblacion.head()


### extracción de dataset comuna, nos trae datos precisos con ID, se suman para realizar al dashboard, no necesitan transformación, se manejara en excel de tal manera que sea compatible para cargar en el database

In [2]:
import pandas as pd

# URL del archivo Excel
url = "https://cdn.buenosaires.gob.ar/datosabiertos/datasets/ministerio-de-educacion/comunas/comunas.xlsx"

# Leer el archivo Excel y convertirlo en un DataFrame
comuna = pd.read_excel(url)

# Mostrar las primeras filas del DataFrame para verificar la carga de datos
comuna.head()


,ID,OBJETO,COMUNAS,BARRIOS,PERIMETRO,AREA
0,1,LIMITE COMUNAL,2,RECOLETA,21452.838648,6.317265e+06
1,2,LIMITE COMUNAL,5,ALMAGRO - BOEDO,12323.432479,6.660603e+06
2,3,LIMITE COMUNAL,6,CABALLITO,10990.964471,6.851029e+06
3,4,LIMITE COMUNAL,7,FLORES - PARQUE CHACABUCO,17972.257870,1.242290e+07
4,5,LIMITE COMUNAL,9,LINIERS - MATADEROS - PARQUE AVELLANEDA,21411.738344,1.650531e+07
